<a href="https://colab.research.google.com/github/ehpub/KFQ_DL_2023-/blob/main/2023_kfq_%EB%94%A5%EB%9F%AC%EB%8B%9D_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 교재: https://www.gilbut.co.kr/book/view?bookcode=BN003496

In [1]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2023-07-26 02:22:54--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  17.4MB/s    in 8.2s    

2023-07-26 02:23:03 (9.75 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



참고 교재: https://www.gilbut.co.kr/book/view?bookcode=BN003496

In [2]:
!tar -xf aclImdb_v1.tar.gz

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models

In [4]:
dataset = keras.utils.text_dataset_from_directory(
    directory = 'aclImdb',
    label_mode = None,
    batch_size=256
)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x,'<br />',' '))

Found 100006 files belonging to 1 classes.


In [5]:
for batch in dataset:
  print(len(batch))
  for text in batch:
    print(text)
    break
  break

256
tf.Tensor(b'I admit that when I first heard of this movie starring Sofi\'a , I wanted to see Sofi\'a topless or in lace nightgowns..but that changed when I watched this movie.. I was familiar with Roselyn\'s work (Boat Trip,Rush Hour 2)and this was the 1st time I saw Jaci. Like with most of the fans of this movie,I fairly enjoyed it,as a guy -I enjoyed the eye-candy..Especially Sofi\'a,even though she emulated that "cuchi-cuchi" Charo style,with her comical antics and Sofi\'a over doing her hot and sexy colombian accent!! Because this movie was directed by a woman,this was a light screwball comedy..the story was simple,the comedy tame,lot of silly clich\xc3\xa9\'s(especially for Sofi\'a)and the acting was OK,though I was surprised at Sofi\'a\'s comedy timing,since she never had any training at acting!!I don\'t know if this movie would have been better if a man directed this,his perspective.But because I enjoyed watching Sofi\'a working off on the other Latina\'s,in all their sexies

In [6]:
from keras import layers

In [7]:
sequence_length = 100
vocab_size=15000
tv = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length=sequence_length
)
tv.adapt(dataset)

In [8]:
for batch in dataset:
  for text in batch:
    print(text)
    vs = tv(text)
    print(vs)
    break
  break

tf.Tensor(b'The subject is certainly compelling: a group of people take their love of gaming one step further by creating a fake medieval world full of warriors, kings, princes and castles. Wargaming is an interesting phenomena that delves into our collective need to "escape" from reality and the sometimes mundaneness of our existence -- something almost everyone can relate to. The characters are the predictable mix of Lord of the Rings nerds and Star Trek enthusiasts. That\'s enough to get most people to watch. However, very quickly the film turns into an insider\'s view of wargaming with an almost stereotypical thumbing of the nose to viewers who "don\'t get it". The filmmakers seem to take the subject of wargaming, and this particular one, waaaaay too seriously rather than once in awhile recognizing the humor and fun in making a film about adults drssing up in medieval gear and pounding each other with foam swords. It\'s pretty hard for anyone who doesn\'t sit on their computer for 

In [9]:
for batch in dataset:
  vs = tv(batch)
  x = vs[:,:-1]
  y = vs[:,1:]
  break
print(vs[0])
print(x[0])
print(y[0])

tf.Tensor(
[1183    2   62  285   10 1632   53   11   17   35 2651   13   84 5693
 3603   13    8    9  802   86  176   86   80   21 1288   87 1502 5693
 3603   43  411   10  384   27  318    6   26   37 1015   18    1   12
    1  664    3    1 2204    1   10  103   27  140 1319   15 1207    3
    2  693    5    2   17 2647  248    4   72    1  221   36   98    1
   54 7462  153   10  195   11 1404   17   13    1  857    3   11    7
  560   35   69  106    4 1404  533    2  384  486    8   47   23    1
  719    2], shape=(100,), dtype=int64)
tf.Tensor(
[1183    2   62  285   10 1632   53   11   17   35 2651   13   84 5693
 3603   13    8    9  802   86  176   86   80   21 1288   87 1502 5693
 3603   43  411   10  384   27  318    6   26   37 1015   18    1   12
    1  664    3    1 2204    1   10  103   27  140 1319   15 1207    3
    2  693    5    2   17 2647  248    4   72    1  221   36   98    1
   54 7462  153   10  195   11 1404   17   13    1  857    3   11    7
  560   35   69

In [10]:
def prepare_lm_dataset(text_batch):
  vs = tv(text_batch)
  x = vs[:,:-1]
  y = vs[:,1:]
  return x,y

In [11]:
lm_dataset = dataset.map(prepare_lm_dataset,num_parallel_calls=4)

In [12]:
embed_dim = 256
latent_dim = 2048
num_heads = 2

In [13]:
class PositionalEmbedding(layers.Layer):
  def __init__(self,sequence_length,input_dim,output_dim,**kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(input_dim,output_dim)
    self.position_embeddings = layers.Embedding(sequence_length,output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim
  def call(self,inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0,limit=length,delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions
  def compute_mask(self,inputs,mask=None):
    return tf.math.not_equal(inputs,0)
  def get_config(self):
    config = super(PositionalEmbedding,self).get_config()
    config.update(
        {
            "output_dim":self.output_dim,
            "sequence_length":self.sequence_length,
            "input_dim":self.input_dim,
        }
    )
    return config

In [14]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)



In [15]:
inputs = keras.Input(shape=(None,),dtype='int64')
x = PositionalEmbedding(sequence_length,vocab_size,embed_dim)(inputs)
x = TransformerDecoder(embed_dim,latent_dim,num_heads)(x,x)
outputs = layers.Dense(vocab_size,activation='softmax')(x)
model = keras.Model(inputs,outputs)
model.compile(loss = keras.losses.sparse_categorical_crossentropy)

In [16]:
import numpy as np

In [17]:
tokens_index = dict(enumerate(tv.get_vocabulary()))

In [18]:
tokens_index

{0: '',
 1: '[UNK]',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'in',
 9: 'it',
 10: 'i',
 11: 'this',
 12: 'that',
 13: 'was',
 14: 'as',
 15: 'with',
 16: 'for',
 17: 'movie',
 18: 'but',
 19: 'film',
 20: 'on',
 21: 'you',
 22: 'not',
 23: 'are',
 24: 'his',
 25: 'have',
 26: 'be',
 27: 'he',
 28: 'one',
 29: 'its',
 30: 'at',
 31: 'all',
 32: 'by',
 33: 'an',
 34: 'they',
 35: 'from',
 36: 'who',
 37: 'so',
 38: 'like',
 39: 'or',
 40: 'just',
 41: 'her',
 42: 'about',
 43: 'has',
 44: 'if',
 45: 'out',
 46: 'some',
 47: 'there',
 48: 'what',
 49: 'good',
 50: 'when',
 51: 'more',
 52: 'very',
 53: 'up',
 54: 'no',
 55: 'even',
 56: 'my',
 57: 'would',
 58: 'she',
 59: 'time',
 60: 'their',
 61: 'which',
 62: 'only',
 63: 'really',
 64: 'story',
 65: 'see',
 66: 'were',
 67: 'can',
 68: 'had',
 69: 'me',
 70: 'than',
 71: 'well',
 72: 'much',
 73: 'we',
 74: 'been',
 75: 'get',
 76: 'bad',
 77: 'into',
 78: 'also',
 79: 'great',
 80: 'do',
 81: 'other',
 82: '

In [19]:
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

In [20]:
class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = tv([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

In [21]:
prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [22]:
model.fit(lm_dataset, epochs=3, callbacks=[text_gen_callback])

Epoch 1/3
391/391 [==============================] - ETA: 0s - loss: 5.9246== Generating with temperature 0.2
This movie movie has that a real lot and of li yourself rather after i half think of about a this lovely only at good all for up all sure the the hell old of cells the sequences delightful was film ok was reading good a brief cinema scenery who after hasnt
== Generating with temperature 0.5
This movie film one forgettable of positive the comments plot and just cary could dreamy feel all for my ten way shows all me different fuss iran and man generally it harvest becomes better [UNK] [UNK] through michael their values [UNK] and both all answer in introduces the it rings much i
== Generating with temperature 0.7
This movie film performers movie and were the laughable next it hang is at no theyre sense released whoever in that place i with mean its id compassion ever i seen watched and the i movies think him thats in my the third way movie to is sibling a on short voodoo
== Genera